## Importação dos dados

In [ ]:
# Leitura de dados
import sys
import subprocess
import pkg_resources

if {'pandas'} not in {pkg.key for pkg in pkg_resources.working_set}:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install','pandas'], stdout=subprocess.DEVNULL)

if {'numpy'} not in {pkg.key for pkg in pkg_resources.working_set}:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install','numpy'], stdout=subprocess.DEVNULL)

if {'matplotlib'} not in {pkg.key for pkg in pkg_resources.working_set}:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install','matplotlib'], stdout=subprocess.DEVNULL)

if {'seaborn'} not in {pkg.key for pkg in pkg_resources.working_set}:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install','seaborn'], stdout=subprocess.DEVNULL)

if {'sklearn'} not in {pkg.key for pkg in pkg_resources.working_set}:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install','sklearn'], stdout=subprocess.DEVNULL)

import matplotlib.pyplot as plt 
import os
import pickle
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

In [ ]:
base = pd.read_csv('../data/sih_aih_env_2011_2020_pneucom.csv',sep=',')

# https://raw.githubusercontent.com/lslbastos/ibge_population_projection/main/output/data_pop_ibge_proj_2010-2016.csv

### População de Referencia (por faixa etária e ano no RJ)

In [ ]:
pop_ref = pd.read_csv('https://raw.githubusercontent.com/lslbastos/ibge_population_projection/main/output/data_pop_ibge_proj_2010-2016.csv',sep=',')

In [ ]:
pop_ref.head()

In [ ]:
pop_ref['sex'].unique()

In [ ]:
pop_ref = pop_ref[(pop_ref['age']!='TOTAL') & (pop_ref['uf']=='BR') & (pop_ref['sex']=='all') & (pop_ref['year'].isin(range(2011,2020)))]

In [ ]:
pop_ref['age'] = pop_ref['age'].astype(int)

In [ ]:
pop_ref.head()

In [ ]:
idadeWHO = base.groupby(['idade_grupo_who','idade_real_anos']).agg({'id':'count'}).reset_index()
idadeWHO.drop('id',axis=1,inplace=True)
idadeWHO

In [ ]:
pop_ref = pd.merge(pop_ref,idadeWHO, how='left', left_on='age',right_on='idade_real_anos')

In [ ]:
pop_ref

In [ ]:
pop_ref[pop_ref['year']==2011]

In [ ]:
pop_ref = pop_ref.groupby(['year','idade_grupo_who']).agg({'population':'sum'}).reset_index()
pop_ref

### População de Referência (por sexo e ano no RJ)

In [ ]:
pop_ref_by_sex = pd.read_csv('https://raw.githubusercontent.com/lslbastos/ibge_population_projection/main/output/data_pop_ibge_proj_2010-2016.csv',sep=',')

pop_ref_by_sex = pop_ref_by_sex[(pop_ref_by_sex['age']!='TOTAL') & (pop_ref_by_sex['uf']=='BR') & (pop_ref_by_sex['year'].isin(range(2011,2020)))]
pop_ref_by_sex['age'] = pop_ref_by_sex['age'].astype(int)
pop_ref_by_sex = pop_ref_by_sex[pop_ref_by_sex['age']>=20].groupby(['sex','year']).agg({'population':'sum'}).reset_index()
pop_ref_by_sex.columns

### Base de Dados do CID-10

In [ ]:
subprocess.check_call([sys.executable, '-m', 'pip', 'install','openpyxl'], stdout=subprocess.DEVNULL)

In [ ]:
#cid10 = pd.read_csv('cids_diag_ahrq_css_br - Copy.csv',sep=',') 

cid10 = pd.read_excel ('../data/cids_diag_ahrq_css_br_v5.xlsx')

In [ ]:
cid10

In [ ]:
#cid10 = cid10[['icd10_br','cid_10_gen_categ','cid_10_det_categ','ccs_lvl_2_label','class_abritrario']]

### Base de Dados do CNES

In [ ]:
cnes = pd.read_csv('../data/cnes_estabelecimentos.csv',sep=';',encoding='unicode_escape')
#cnes = pd.read_csv('estabelecimentos-330000.csv',sep=';')

In [ ]:
cnes.shape

In [ ]:
cnes.head()

In [ ]:
cnes_prof = pd.read_csv('../data/profissionais-330000.csv',sep=';')

In [ ]:
cnes_prof.head()

In [ ]:
cnes_pf = cnes_prof.groupby('CNES').agg({'NOME':'count'}).reset_index()
cnes_pf.columns = ['cnes','qnt_prof']

In [ ]:
cnes = pd.merge(cnes,cnes_pf,how='left',left_on='CO_CNES',right_on='cnes')

In [ ]:
cnes.head()

## Filtrando a Base

In [ ]:
base.head()

In [ ]:
base = base[base['ano_inter'].isin(range(2011,2020))]

In [ ]:
base = pd.merge(base,cid10,how='left',left_on='diag_princ_trim_upper',right_on='icd10_br')

In [ ]:
base.head()

In [ ]:
base = pd.merge(base,cnes,how='left',left_on='cnes',right_on='CO_CNES')

In [ ]:
base.head()

In [ ]:
#base['remover'] = base['diag_princ_trim_upper'].apply(lambda x: 1 if x.startswith('F') else (2 if x.startswith('O') else (3 if x.startswith('P') else (4 if x.startswith('Q') else 0 ) ) ) )
# F # ICD-10 CHAPTER V - Mental and behavioral disorders
# O # ICD-10 CHAPTER XV - Pregnancy, childbirth and the puerperium
# P # ICD-10 CHAPTER XVI - Certain conditions originating in the perinatal period
# Q # ICD-10 CHAPTER XVII - Congenital malformations, deformations and chromosomal abnormalities

#base = base[base['class_abritrario'].isin(['Infeccions', 'Cronic-Deg', 'Injuries', 'Neoplasms'])]

#base['remover'] = base['diag_princ_trim_upper'].apply(lambda x: 0 if x.startswith('J10') else (0 if x.startswith('J12') else (0 if x.startswith('J13') else (0 if x.startswith('J14') else (0 if x.startswith('J15') else (0 if x.startswith('J16') else (0 if x.startswith('J17') else (0 if x.startswith('J18') else 1 ) ) ) ) ) ) ) )

In [ ]:
#base[base['remover'] == 0]['diag_princ_trim_upper'].head()

In [ ]:
#base = base[base['remover'] == 0]

In [ ]:
base['classificacao'] = base['idade_grupo_who'].apply(lambda x: 1 if x in ['60-64','65-69','70-74','75-79','80-84','85+'] else 0)

#y = ['60-64']
#y.isin(['60-64','65-69','70-74','75-79','80-84','85+'])

In [ ]:
base.shape

In [ ]:
base['CO_NATUREZA_JUR'].unique()

In [ ]:
base['nat_jur'] = base['CO_NATUREZA_JUR'].apply(lambda x: 'Administração Pública' if x in range(1000,2000) else ('Entidades Empresariais' if x in range(2000,3000) else ('Entidades sem Fins Lucrativos' if x in range(3000,4000) else ('Pessoas Físicas' if x in range(4000,5000) else 'Organizações Internacionais e Outras Instituições Extraterritoriais'))))


In [ ]:
base[['CO_NATUREZA_JUR','nat_jur']].head()

In [ ]:
base['nat_jur'].unique()

In [ ]:
# Qnt de idosos
base[base['idade_real_anos'] >= 60]['id'].count()

In [ ]:
# Qnt de não idosos
base[base['idade_real_anos'] < 60]['id'].count()

In [ ]:
base.columns

In [ ]:
pd.set_option('display.max_columns', None)
base.head()

In [ ]:
#base.drop(['CO_CNES','CO_UNIDADE','CO_UF','CO_IBGE','NU_CNPJ_MANTENEDORA','NO_RAZAO_SOCIAL','NO_FANTASIA','CO_NATUREZA_ORGANIZACAO','DS_NATUREZA_ORGANIZACAO','DS_NATUREZA_ORGANIZACAO','CO_NIVEL_HIERARQUIA','',''])

In [ ]:
base_eld = base[base['classificacao'] == 1]
base_eld.head()

In [ ]:
base_noneld = base[base['classificacao'] == 0]
base_noneld.head()

In [ ]:
# PROPORCAO DA POPULACAO POR FAIXA ETARIA (WHO)

dfWHO = {'age_group': ['0-4', '5-9', '10-14', '15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85+'],
        'world_avg': [8.86, 8.69, 8.60, 8.47, 8.22, 7.93, 7.61, 7.15, 6.59, 6.04, 5.37, 4.55, 3.72, 2.96, 2.21, 1.52, 0.91, 0.60]}
dfWHO = pd.DataFrame(dfWHO)
dfWHO['world_avg_dec'] = dfWHO['world_avg']/100
dfWHO

### Funções auxiliares

In [ ]:
from sklearn import linear_model
subprocess.check_call([sys.executable, '-m', 'pip', 'install','statsmodels'], stdout=subprocess.DEVNULL)
import statsmodels.api as sm

In [ ]:
from patsy import dmatrices
import math

In [ ]:
def aapc(df,expr,name):
    y_train, X_train = dmatrices(expr, df, return_type='dataframe')
    poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
    aapc_value = (math.exp(poisson_training_results.params["ano_inter"]) - 1)*100
    aapc_inf = (math.exp(poisson_training_results.conf_int().loc['ano_inter',0]) - 1)*100
    aapc_sup = (math.exp(poisson_training_results.conf_int().loc['ano_inter',1]) - 1)*100

    data = {'Analise': [name],
        'AAPC': [aapc_value],
        'IC_Inf': [aapc_inf],
        'IC_Sup': [aapc_sup]}
    df_results = pd.DataFrame(data)
    return df_results

In [ ]:
def age_adjust_ML(df,dfWHO,pop_ref):
    
    df.columns = ['ano_inter','idade_grupo_who','num']
    df_merged = pd.merge(pop_ref, df,  how='left', left_on=['year','idade_grupo_who'], right_on=['ano_inter','idade_grupo_who']) # junta a quantidade por ano por faixa etária com a população de referencia
    df_merged['taxa'] = df_merged['num']/df_merged['population'] # calcula a taxa
    result = pd.merge(df_merged, dfWHO,  how='left', left_on=['idade_grupo_who'], right_on=['age_group']) # junta o dataframe com a proporção do WHO
    result['obitos_esp'] = result['taxa']*result['world_avg_dec'] # calcula a proporção da quantidade esperados
    df_result = result.groupby(['year']).agg({'num':'sum','obitos_esp':'sum','population':'sum','world_avg_dec':'sum'}).reset_index() # agrupa por ANO
    # tx = obts esp / pop
    #somar as faixas etarias do ano e dividir pela pop de ref total
    df_result['taxa_ajustada'] = df_result['obitos_esp']/df_result['world_avg_dec']
    df_result['taxa_ajustada_100mil'] = (df_result['taxa_ajustada']*100000).apply(np.ceil)
    df_result['taxa_bruta_100mil'] = (df_result['num']/df_result['population'])*100000
    df_result['taxa_ajd_qnt'] = df_result['obitos_esp']*df_result['population'] # calcula a quantidade para a populacao de referencia
    df_result['taxa_ajd_qnt_Y'] = df_result['taxa_ajd_qnt'].apply(np.ceil) # arredonda para ter um numero inteiro (count data)
    #df_result = result.groupby(['year']).agg({'taxa_ajd_qnt_Y':'sum','population':'sum'}).reset_index() # agrupa por ANO
    df_result_final = df_result[['year','population','taxa_ajd_qnt_Y','taxa_ajustada','taxa_ajustada_100mil','taxa_bruta_100mil']]
    return df_result_final

In [ ]:
def age_adjust(df,dfWHO,pop_ref):
    
    df.columns = ['ano_inter','idade_grupo_who','num']
    df_merged = pd.merge(pop_ref, df,  how='left', left_on=['year','idade_grupo_who'], right_on=['ano_inter','idade_grupo_who']) # junta a quantidade por ano por faixa etária com a população de referencia
    df_merged['taxa'] = df_merged['num']/df_merged['population'] # calcula a taxa
    result = pd.merge(df_merged, dfWHO,  how='left', left_on=['idade_grupo_who'], right_on=['age_group']) # junta o dataframe com a proporção do WHO
    result['obitos_esp'] = result['taxa']*result['world_avg_dec'] # calcula a proporção da quantidade esperados
    df_result = result.groupby(['year']).agg({'num':'sum','obitos_esp':'sum','population':'sum','world_avg_dec':'sum'}).reset_index() # agrupa por ANO
    # tx = obts esp / pop
    #somar as faixas etarias do ano e dividir pela pop de ref total
    df_result['taxa_ajustada'] = df_result['obitos_esp']/df_result['world_avg_dec']
    df_result['taxa_ajustada_100mil'] = (df_result['taxa_ajustada']*100000).apply(np.ceil)
    df_result['taxa_bruta_100mil'] = (df_result['num']/df_result['population'])*100000
    df_result['taxa_ajd_qnt'] = df_result['obitos_esp']*df_result['population'] # calcula a quantidade para a populacao de referencia
    df_result['taxa_ajd_qnt_Y'] = df_result['taxa_ajd_qnt'].apply(np.ceil) # arredonda para ter um numero inteiro (count data)
    #df_result = result.groupby(['year']).agg({'taxa_ajd_qnt_Y':'sum','population':'sum'}).reset_index() # agrupa por ANO
    df_result_final = df_result[['year','population','taxa_ajd_qnt_Y','taxa_ajustada_100mil']]
    return df_result_final

In [ ]:
#def aapc_offset(df,expr,name):
#    y_train, X_train = dmatrices(expr, df, return_type='dataframe')
#    poisson_training_results = sm.GLM(y_train, X_train, offset=X_train['np.log(population)'], family=sm.families.Poisson()).fit()
#    #print(poisson_training_results.summary())
#    aapc_value = (math.exp(poisson_training_results.params["year"]) - 1)*100
#    aapc_inf = (math.exp(poisson_training_results.conf_int().loc['year',0]) - 1)*100
#    aapc_sup = (math.exp(poisson_training_results.conf_int().loc['year',1]) - 1)*100
#
#    data = {'Analise': [name],
#        'AAPC': [aapc_value],
#        'IC_Inf': [aapc_inf],
#        'IC_Sup': [aapc_sup]}
#    df_results = pd.DataFrame(data)
#    return df_results

In [ ]:
def aapc_offset(df,expr,name):
    y_train, X_train = dmatrices(expr, df, return_type='dataframe')
    poisson_training_results = sm.GLM(y_train, X_train, offset=np.log(df['population']), family=sm.families.Poisson()).fit()
    #print(poisson_training_results.summary())
    aapc_value = (math.exp(poisson_training_results.params["year"]) - 1)*100
    aapc_inf = (math.exp(poisson_training_results.conf_int().loc['year',0]) - 1)*100
    aapc_sup = (math.exp(poisson_training_results.conf_int().loc['year',1]) - 1)*100

    data = {'Analise': [name],
        'AAPC': [aapc_value],
        'IC_Inf': [aapc_inf],
        'IC_Sup': [aapc_sup]}
    df_results = pd.DataFrame(data)
    return df_results

In [ ]:
def registros_ano(lst_dfs,lst_nomes,col_ano):

    df_first = lst_dfs[0].copy()
    nome = lst_nomes[0]

    if col_ano == 'year':
        df_first.drop('population',axis=1,inplace=True)
        df_first.drop('taxa_ajd_qnt_Y',axis=1,inplace=True)


    df_first = df_first.set_index(col_ano).T
    df_first['Analise'] = nome
    df_first['delta'] = ((df_first[2020] - df_first[2011])/df_first[2011])*100

    for i in range(1,len(lst_dfs)):
        df = lst_dfs[i].copy()
        nome = lst_nomes[i]

        if col_ano == 'year':
            df.drop('population',axis=1,inplace=True)
            df.drop('taxa_ajd_qnt_Y',axis=1,inplace=True)

        df = df.set_index(col_ano).T
        df['Analise'] = nome
        df['delta'] = ((df[2020] - df[2011])/df[2011])*100
    
        df_first = pd.concat([df_first,df])
    
    #df_first.drop(col_ano,axis=1,inplace=True)
    
    return df_first

## Analise Exploratória

### População por sexo

In [ ]:
# Característica da População

pop_total = pop_ref_by_sex[pop_ref_by_sex['sex']=='all']['population'].reset_index()

pop_homens = pop_ref_by_sex[pop_ref_by_sex['sex']=='male']['population'].reset_index()
pop_mulheres = pop_ref_by_sex[pop_ref_by_sex['sex']=='female']['population'].reset_index()

r = [0,1,2,3,4,5,6,7,8]
g1_bars_bottom = round((pop_homens/pop_total)*100)['population']
g1_bars_top = round((pop_mulheres/pop_total)*100)['population']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax1.bar(r, g1_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
p2 = ax1.bar(r, g1_bars_top, bottom=g1_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Population", fontweight='bold')
ax1.bar_label(p1, label_type='center', fontweight='bold')
ax1.bar_label(p2, label_type='center', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.35, 0.01), ncol=2, labelspacing=0.)
plt.savefig('../images/population_by_sex.png',dpi=1200)
plt.show()

### Admissões por sexo

In [ ]:
# Característica da População

# Sexo: 1-> masc, 3-> fem

admissoes_total = base.groupby(['ano_inter']).agg({'id':'count'})

admissoes_homens = base[base['sexo'] == 1].groupby(['ano_inter']).agg({'id':'count'}) 
admissoes_mulheres = base[base['sexo'] == 3].groupby(['ano_inter']).agg({'id':'count'})

r = [0,1,2,3,4,5,6,7,8]
g1_bars_bottom = round((admissoes_homens/admissoes_total)*100)['id']
g1_bars_top = round((admissoes_mulheres/admissoes_total)*100)['id']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax1.bar(r, g1_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
p2 = ax1.bar(r, g1_bars_top, bottom=g1_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Admissions", fontweight='bold')
ax1.bar_label(p1, label_type='center', fontweight='bold')
ax1.bar_label(p2, label_type='center', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.35, 0.01), ncol=2, labelspacing=0.)
plt.savefig('../images/admissions_by_sex.png',dpi=1200)
plt.show()

### Mortes por sexo

In [ ]:
mortes_total = base.groupby(['ano_inter']).agg({'morte':'sum'})

mortes_homens =  base[base['sexo'] == 1].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_mulheres =  base[base['sexo'] == 3].groupby(['ano_inter']).agg({'morte':'sum'})

r = [0,1,2,3,4,5,6,7,8]
g3_bars_bottom = round((mortes_homens/mortes_total)*100)['morte']
g3_bars_top = round((mortes_mulheres/mortes_total)*100)['morte']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax1.bar(r, g3_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
p2 = ax1.bar(r, g3_bars_top, bottom=g3_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Deaths", fontweight='bold')
ax1.bar_label(p1, label_type='center', fontweight='bold')
ax1.bar_label(p2, label_type='center', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.35, 0.01), ncol=2, labelspacing=0.)
plt.savefig('../images/deaths_by_sex.png',dpi=1200)
plt.show()


### Admissões por raça/cor

In [ ]:
# Raça/Cor: 1 -> branca, 2 -> preta, 3-> parda, 4 -> amarela, 5 -> indigena, 99 -> sem info

admissoes_brancos = base[base['raca_cor'] == 1].groupby(['ano_inter']).agg({'id':'count'})
admissoes_pretos = base[base['raca_cor'] == 2].groupby(['ano_inter']).agg({'id':'count'})
admissoes_pardos = base[base['raca_cor'] == 3].groupby(['ano_inter']).agg({'id':'count'})
admissoes_amarelos = base[base['raca_cor'] == 4].groupby(['ano_inter']).agg({'id':'count'})
admissoes_indigenas = base[base['raca_cor'] == 5].groupby(['ano_inter']).agg({'id':'count'})
admissoes_seminfo = base[base['raca_cor'] == 99].groupby(['ano_inter']).agg({'id':'count'})


r = [0,1,2,3,4,5,6,7,8]
g2_bars_bottom = np.nan_to_num(round((admissoes_brancos/admissoes_total)*100)['id'])
g2_bars_bottom2 = np.nan_to_num(round((admissoes_pardos/admissoes_total)*100)['id'])
g2_bars_bottom3 = np.nan_to_num(round((admissoes_pretos/admissoes_total)*100)['id'])
g2_bars_bottom4 = np.nan_to_num(round((admissoes_amarelos/admissoes_total)*100)['id'])
g2_bars_bottom5 = np.nan_to_num(round((admissoes_indigenas/admissoes_total)*100)['id'])
g2_bars_top = np.nan_to_num(round((admissoes_seminfo/admissoes_total)*100)['id'])


fig, ax2 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax2.bar(r, g2_bars_bottom, edgecolor='white', color = '#EB6C50' , label='Brancos')
p2 = ax2.bar(r, g2_bars_bottom2, bottom=g2_bars_bottom, edgecolor='white', color = '#BB53F5', label='Pardos')
p3 = ax2.bar(r, g2_bars_bottom3, bottom=g2_bars_bottom+g2_bars_bottom2, edgecolor='white', color = '#57A5DE', label='Pretos')
p4 = ax2.bar(r, g2_bars_bottom4, bottom=g2_bars_bottom+g2_bars_bottom2+g2_bars_bottom3, color = '#53F56C', label='Amarelos')
p5 = ax2.bar(r, g2_bars_bottom5, bottom=g2_bars_bottom+g2_bars_bottom2+g2_bars_bottom3+g2_bars_bottom4, edgecolor='white', color = '#EBD222', label='Indígenas')
p6 = ax2.bar(r, g2_bars_top, bottom=g2_bars_bottom+g2_bars_bottom2+g2_bars_bottom3+g2_bars_bottom4+g2_bars_bottom5, edgecolor='white', color = '#DC143C', label='Sem Informação')
ax2.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax2.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax2.set_ylabel('Proportion', fontweight='bold')
ax2.set_title("Admissions", fontweight='bold')
ax2.bar_label(p1, label_type='center', fontweight='bold')
ax2.bar_label(p2, label_type='center', fontweight='bold')
ax2.bar_label(p3, label_type='center', fontweight='bold')
# ax2.bar_label(p4, label_type='center', fontweight='bold')
# ax2.bar_label(p5, label_type='center', fontweight='bold')
ax2.bar_label(p6, label_type='center', fontweight='bold')
# ax2.set_facecolor("#E5F2A5")
fig.set_facecolor("#E5F2A5")
handles, labels = ax2.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.25, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/admissions_by_color.png',dpi=1200)
plt.show()

### Mortes por raça/cor

In [ ]:
# Raça/Cor: 1 -> branca, 2 -> preta, 3-> parda, 4 -> amarela, 5 -> indigena, 99 -> sem info

mortes_brancos = base[base['raca_cor'] == 1].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_pretos = base[base['raca_cor'] == 2].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_pardos = base[base['raca_cor'] == 3].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_amarelos = base[base['raca_cor'] == 4].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_indigenas = base[base['raca_cor'] == 5].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_seminfo = base[base['raca_cor'] == 99].groupby(['ano_inter']).agg({'morte':'sum'})


r = [0,1,2,3,4,5,6,7,8]
g4_bars_bottom = np.nan_to_num(round((mortes_brancos/mortes_total)*100)['morte'])
g4_bars_bottom2 = np.nan_to_num(round((mortes_pardos/mortes_total)*100)['morte'])
g4_bars_bottom3 = np.nan_to_num(round((mortes_pretos/mortes_total)*100)['morte'])
g4_bars_bottom4 = np.nan_to_num(round((mortes_amarelos/mortes_total)*100)['morte'])
g4_bars_bottom5 = np.nan_to_num(round((mortes_indigenas/mortes_total)*100)['morte'])
g4_bars_top = np.nan_to_num(round((mortes_seminfo/mortes_total)*100)['morte'])


fig, ax2 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax2.bar(r, g4_bars_bottom, edgecolor='white', color = '#EB6C50' , label='Brancos')
p2 = ax2.bar(r, g4_bars_bottom2, bottom=g4_bars_bottom, edgecolor='white', color = '#BB53F5', label='Pardos')
p3 = ax2.bar(r, g4_bars_bottom3, bottom=g4_bars_bottom+g4_bars_bottom2, edgecolor='white', color = '#57A5DE', label='Pretos')
p4 = ax2.bar(r, g4_bars_bottom4, bottom=g4_bars_bottom+g4_bars_bottom2+g4_bars_bottom3, color = '#53F56C', label='Amarelos')
p5 = ax2.bar(r, g4_bars_bottom5, bottom=g4_bars_bottom+g4_bars_bottom2+g4_bars_bottom3+g4_bars_bottom4, edgecolor='white', color = '#EBD222', label='Indígenas')
p6 = ax2.bar(r, g4_bars_top, bottom=g4_bars_bottom+g4_bars_bottom2+g4_bars_bottom3+g4_bars_bottom4+g4_bars_bottom5, edgecolor='white', color = '#DC143C', label='Sem Informação')
ax2.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax2.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax2.set_ylabel('Proportion', fontweight='bold')
ax2.set_title("Deaths", fontweight='bold')
ax2.bar_label(p1, label_type='center', fontweight='bold')
ax2.bar_label(p2, label_type='center', fontweight='bold')
ax2.bar_label(p3, label_type='center', fontweight='bold')
# ax2.bar_label(p4, label_type='center', fontweight='bold')
# ax2.bar_label(p5, label_type='center', fontweight='bold')
ax2.bar_label(p6, label_type='center', fontweight='bold')
# ax2.set_facecolor("#E5F2A5")
fig.set_facecolor("#E5F2A5")
handles, labels = ax2.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.25, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/deaths_by_color.png',dpi=1200)
plt.show()

### Taxa de mortalidade e letalidade

In [ ]:
dfmortes = base.groupby(['ano_inter','idade_grupo_who']).agg({'morte':'sum'}).reset_index()
dfmortalidade = age_adjust_ML(dfmortes,dfWHO,pop_ref)
dfmortalidade

In [ ]:
dflet = base.groupby(['ano_inter','idade_grupo_who']).agg({'id':'count','morte':'sum'}).reset_index()
dflet.columns = ['ano_inter','idade_grupo_who','admissoes','num']
#df_merged = pd.merge(pop_ref, df,  how='left', left_on=['year','idade_grupo_who'], right_on=['ano_inter','idade_grupo_who']) # junta a quantidade por ano por faixa etária com a população de referencia
dflet['taxa'] = dflet['num']/dflet['admissoes'] # calcula a taxa
dfletalidade = pd.merge(dflet, dfWHO,  how='left', left_on=['idade_grupo_who'], right_on=['age_group']) # junta o dataframe com a proporção do WHO
dfletalidade['obitos_esp'] = dfletalidade['taxa']*dfletalidade['world_avg_dec'] # calcula a proporção da quantidade esperados
dfletalidade = dfletalidade.groupby(['ano_inter']).agg({'num':'sum','obitos_esp':'sum','admissoes':'sum','world_avg_dec':'sum'}).reset_index() # agrupa por ANO
# tx = obts esp / pop
#somar as faixas etarias do ano e dividir pela pop de ref total
dfletalidade['taxa_ajustada'] = dfletalidade['obitos_esp']/dfletalidade['world_avg_dec']
dfletalidade['taxa_ajustada_100mil'] = (dfletalidade['taxa_ajustada']*100000).apply(np.ceil)
dfletalidade['taxa_bruta_100mil'] = (dfletalidade['num']/dfletalidade['admissoes'])*100000
dfletalidade

In [ ]:
tx_mort = round(dfmortalidade['taxa_ajustada']*100,4)[0:9]
tx_let = round((dfletalidade['taxa_ajustada'])*100,4)

r = [0,1,2,3,4,5,6,7,8]

fig, (ax1,ax2) = plt.subplots(1,2,sharex=True,)
p1 = ax1.plot(r, tx_mort, marker='o', markerfacecolor='#380282', markersize=4, color = '#380282' , label='Mortality rate')
p2 = ax2.plot(r, tx_let, marker='o', markerfacecolor='#380282', markersize=4, color = '#DC143C' , label='Lethality rate')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,0.01,0.02,0.03,0.04,0.05],['0%','0.01%','0.02%','0.03%','0.04%','0.05%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Mortality Rate", fontweight='bold')
a = [1,-1,1,-1,1,-1,1,-1,1,-1]
ax2.set_yticks([0,10,20,30,40,50],['0%','10%','20%','30%','40%','50%'])
ax2.set_ylabel('Proportion', fontweight='bold')
ax2.set_title("Lethality Rate", fontweight='bold')

for i in range(len(tx_mort)):
    ax1.text(r[i], tx_mort[i]+(a[i]*0.0017), "%.4f" %tx_mort[i], ha="center", fontweight='bold')
    ax2.text(r[i], tx_let[i]+1, "%d" %tx_let[i], ha="center", fontweight='bold')
    
fig.set_facecolor("#E5F2A5")

fig.set_figheight(6)
fig.set_figwidth(12)
plt.savefig('../images/mortality_ratio.png',dpi=1200)
plt.show()



### Valores faltantes na Base

In [ ]:
na=(base.shape[0]-base[base.columns].count()).to_frame().reset_index().rename(columns={'index':'COLUNA',0:'NAs'})
na

In [ ]:
fig, ax = plt.subplots(figsize=(10,15))
sns.barplot(y='COLUNA',x='NAs',data=na,ax=ax).set_title('VALORES NULOS', fontsize=15)
#fig.savefig("imagens2\\nulos.png", dpi=None, facecolor='w', edgecolor='w',
#        orientation='portrait', papertype=None, format=None,
#        transparent=False, bbox_inches=None, pad_inches=0.1,
#        frameon=None, metadata=None)

In [ ]:
base.drop(['NU_CNPJ_MANTENEDORA','CO_NATUREZA_ORGANIZACAO','DS_NATUREZA_ORGANIZACAO','DS_NATUREZA_ORGANIZACAO','CO_NIVEL_HIERARQUIA','DS_NIVEL_HIERARQUIA','CO_ESFERA_ADMINISTRATIVA','DS_ESFERA_ADMINISTRATIVA','NU_TELEFONE','NU_LATITUDE','NU_LONGITUDE','NU_CNPJ','NO_EMAIL','CO_MOTIVO_DESAB','cnes_y','qnt_prof'],axis=1,inplace=True)

In [ ]:
base.columns

### Admissoes por tipo de hospital

In [ ]:
# ADMISSOES POR TIPO DE HOSPITAL

admissoes_total = base.groupby(['ano_inter']).agg({'id':'count'})

admissoes_hosppublico = base[base['nat_jur'] == 'Administração Pública'].groupby(['ano_inter']).agg({'id':'count'}) 
admissoes_hosppart = base[base['nat_jur'] == 'Entidades Empresariais'].groupby(['ano_inter']).agg({'id':'count'})
admissoes_hospong = base[base['nat_jur'] == 'Entidades sem Fins Lucrativos'].groupby(['ano_inter']).agg({'id':'count'})


r = [0,1,2,3,4,5,6,7,8]
g1_bars_bottom = round((admissoes_hosppublico/admissoes_total)*100)['id']
g1_bars_mid = round((admissoes_hospong/admissoes_total)*100)['id']
g1_bars_top = round((admissoes_hosppart/admissoes_total)*100)['id']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax1.bar(r, g1_bars_bottom, edgecolor='white', color = 'darkgreen' , label='Hospital Público')
p2 = ax1.bar(r, g1_bars_mid, bottom=g1_bars_bottom, edgecolor='white', color = 'orange' , label='Hospital sem Fins Lucrativos')
p3 = ax1.bar(r, g1_bars_top, bottom=g1_bars_bottom+g1_bars_mid, edgecolor='white', color = 'navy', label='Hospital Privado')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Admissions", fontweight='bold')
ax1.bar_label(p1, label_type='center', fontweight='bold')
ax1.bar_label(p2, label_type='center', fontweight='bold')
ax1.bar_label(p3, label_type='center', fontweight='bold',color='white')


fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.2, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/admissions_by_hospital.png',dpi=1200)
plt.show()

### Mortes por tipo de hospital

In [ ]:
mortes_total = base.groupby(['ano_inter']).agg({'morte':'sum'})

mortes_hosppublico =  base[base['nat_jur'] == 'Administração Pública'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_hosppart =  base[base['nat_jur'] == 'Entidades Empresariais'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_hospong =  base[base['nat_jur'] == 'Entidades sem Fins Lucrativos'].groupby(['ano_inter']).agg({'morte':'sum'})

r = [0,1,2,3,4,5,6,7,8]
g3_bars_bottom = round((mortes_hosppublico/mortes_total)*100)['morte']
g3_bars_mid = round((mortes_hospong/mortes_total)*100)['morte']
g3_bars_top = round((mortes_hosppart/mortes_total)*100)['morte']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax1.bar(r, g3_bars_bottom, edgecolor='white', color = 'darkgreen' , label='Hospital Público')
p2 = ax1.bar(r, g3_bars_mid, bottom=g3_bars_bottom, edgecolor='white', color = 'orange', label='Hospital Sem Fins Lucrativos')
p3 = ax1.bar(r, g3_bars_top, bottom=g3_bars_bottom+g3_bars_mid, edgecolor='white', color = 'navy', label='Hospital Privado')
ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Deaths", fontweight='bold')
ax1.bar_label(p1, label_type='center', fontweight='bold')
ax1.bar_label(p2, label_type='center', fontweight='bold')
ax1.bar_label(p3, label_type='center', fontweight='bold',color='white')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.2, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/deaths_by_hospital.png',dpi=1200)
plt.show()

### Admissoes por Regiao

In [ ]:
base['regiao'].unique()

In [ ]:
admissoes_S = base[base['regiao'] == 'S'].groupby(['ano_inter']).agg({'id':'count'})
admissoes_N = base[base['regiao'] == 'N'].groupby(['ano_inter']).agg({'id':'count'})
admissoes_NE = base[base['regiao'] == 'NE'].groupby(['ano_inter']).agg({'id':'count'})
admissoes_SE = base[base['regiao'] == 'SE'].groupby(['ano_inter']).agg({'id':'count'})
admissoes_CO = base[base['regiao'] == 'CO'].groupby(['ano_inter']).agg({'id':'count'})


r = [0,1,2,3,4,5,6,7,8]
g2_bars_bottom = np.nan_to_num(round((admissoes_S/admissoes_total)*100)['id'])
g2_bars_bottom2 = np.nan_to_num(round((admissoes_N/admissoes_total)*100)['id'])
g2_bars_bottom3 = np.nan_to_num(round((admissoes_NE/admissoes_total)*100)['id'])
g2_bars_bottom4 = np.nan_to_num(round((admissoes_SE/admissoes_total)*100)['id'])
g2_bars_bottom5 = np.nan_to_num(round((admissoes_CO/admissoes_total)*100)['id'])


fig, ax2 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax2.bar(r, g2_bars_bottom, edgecolor='white', color = '#442288' , label='Sul')
p2 = ax2.bar(r, g2_bars_bottom2, bottom=g2_bars_bottom, edgecolor='white', color = '#6CA2EA', label='Norte')
p3 = ax2.bar(r, g2_bars_bottom3, bottom=g2_bars_bottom+g2_bars_bottom2, edgecolor='white', color = '#FED23F', label='Nordeste')
p4 = ax2.bar(r, g2_bars_bottom4, bottom=g2_bars_bottom+g2_bars_bottom2+g2_bars_bottom3, color = '#B5D33D', label='Sudeste')
p5 = ax2.bar(r, g2_bars_bottom5, bottom=g2_bars_bottom+g2_bars_bottom2+g2_bars_bottom3+g2_bars_bottom4, edgecolor='white', color = '#EB7D5B', label='Centro Oeste')
ax2.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax2.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax2.set_ylabel('Proportion', fontweight='bold')
ax2.set_title("Admissions", fontweight='bold')
ax2.bar_label(p1, label_type='center', fontweight='bold')
ax2.bar_label(p2, label_type='center', fontweight='bold')
ax2.bar_label(p3, label_type='center', fontweight='bold')
ax2.bar_label(p4, label_type='center', fontweight='bold')
ax2.bar_label(p5, label_type='center', fontweight='bold')
# ax2.set_facecolor("#E5F2A5")
fig.set_facecolor("#E5F2A5")
handles, labels = ax2.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.25, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/admissions_by_REGION.png',dpi=1200)
plt.show()

### Mortes por Regiao

In [ ]:
# Raça/Cor: 1 -> branca, 2 -> preta, 3-> parda, 4 -> amarela, 5 -> indigena, 99 -> sem info

mortes_S = base[base['regiao'] == 'S'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_N = base[base['regiao'] == 'N'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_NE = base[base['regiao'] == 'NE'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_SE = base[base['regiao'] == 'SE'].groupby(['ano_inter']).agg({'morte':'sum'})
mortes_CO = base[base['regiao'] == 'CO'].groupby(['ano_inter']).agg({'morte':'sum'})


r = [0,1,2,3,4,5,6,7,8]
g4_bars_bottom = np.nan_to_num(round((mortes_S/mortes_total)*100)['morte'])
g4_bars_bottom2 = np.nan_to_num(round((mortes_N/mortes_total)*100)['morte'])
g4_bars_bottom3 = np.nan_to_num(round((mortes_NE/mortes_total)*100)['morte'])
g4_bars_bottom4 = np.nan_to_num(round((mortes_SE/mortes_total)*100)['morte'])
g4_bars_bottom5 = np.nan_to_num(round((mortes_CO/mortes_total)*100)['morte'])


fig, ax2 = plt.subplots(1,1,sharex=True,sharey=True,)
p1 = ax2.bar(r, g4_bars_bottom, edgecolor='white', color = '#442288' , label='Sul')
p2 = ax2.bar(r, g4_bars_bottom2, bottom=g4_bars_bottom, edgecolor='white', color = '#6CA2EA', label='Norte')
p3 = ax2.bar(r, g4_bars_bottom3, bottom=g4_bars_bottom+g4_bars_bottom2, edgecolor='white', color = '#FED23F', label='Nordeste')
p4 = ax2.bar(r, g4_bars_bottom4, bottom=g4_bars_bottom+g4_bars_bottom2+g4_bars_bottom3, color = '#B5D33D', label='Sudeste')
p5 = ax2.bar(r, g4_bars_bottom5, bottom=g4_bars_bottom+g4_bars_bottom2+g4_bars_bottom3+g4_bars_bottom4, edgecolor='white', color = '#EB7D5B', label='Centro Oeste')
ax2.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax2.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax2.set_ylabel('Proportion', fontweight='bold')
ax2.set_title("Deaths", fontweight='bold')
ax2.bar_label(p1, label_type='center', fontweight='bold')
ax2.bar_label(p2, label_type='center', fontweight='bold')
ax2.bar_label(p3, label_type='center', fontweight='bold')
ax2.bar_label(p4, label_type='center', fontweight='bold')
ax2.bar_label(p5, label_type='center', fontweight='bold')
# ax2.set_facecolor("#E5F2A5")
fig.set_facecolor("#E5F2A5")
handles, labels = ax2.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.25, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/deaths_by_region.png',dpi=1200)
plt.show()

### LOS de pacientes em Uti por sexo

In [ ]:
losuti_total = base[base['uti']==1].groupby(['ano_inter','sexo']).agg({'los_uti':'mean'}).reset_index()
losuti_total['Sexo'] = losuti_total['sexo'].apply(lambda x: 'Homem' if x == 1 else "Mulher")


losuti_homens =  base[(base['sexo'] == 1)&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_mulheres =  base[(base['sexo'] == 3)&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()

r = [0,2,4,6,8,10,12,14,16]
r2 = [x+0.6 for x in r]
#g3_bars_bottom = round((losuti_homens/losuti_total)*100)['los_uti']
#g3_bars_top = round((losuti_mulheres/losuti_total)*100)['los_uti']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
#p1 = ax1.bar(r, g3_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
#p2 = ax1.bar(r, g3_bars_top, bottom=g3_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')

p1 = ax1.bar(r, round(losuti_homens['los_uti'],1), color = '#380282', width = 0.4, label='Homens')
p2 = ax1.bar(r2, round(losuti_mulheres['los_uti'],1), color = '#DC143C',width = 0.4, label='Mulheres')
#ax.bar(X + 0.50, data[2], color = 'r', width = 0.25)

#p1 = sns.barplot(data=losuti_total, x='ano_inter',y='los_uti',hue='Sexo')

#p2 = sns.barplot(data=losuti_mulheres, x='ano_inter',y='los_uti',color = '#DC143C' , label='Homens')

ax1.set_xticks([x+0.3 for x in r], ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
#ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Days', fontweight='bold')
ax1.set_xlabel('Years', fontweight='bold')

ax1.set_title("Average LOS Uti", fontweight='bold')
ax1.bar_label(p1, label_type='edge', fontweight='bold')
ax1.bar_label(p2, label_type='edge', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.35, 0.01), ncol=2, labelspacing=0.)
plt.savefig('../images/los_by_sex.png',dpi=1200)
plt.show()

### LOS UTI por tipo de hospital

In [ ]:
#losuti_total = base[base['uti']==1].groupby(['ano_inter','sexo']).agg({'los_uti':'mean'}).reset_index()
#losuti_total['Sexo'] = losuti_total['sexo'].apply(lambda x: 'Homem' if x == 1 else "Mulher")

#mortes_hosppublico =  base[base['nat_jur'] == 'Administração Pública'].groupby(['ano_inter']).agg({'morte':'sum'})
#mortes_hosppart =  base[base['nat_jur'] == ].groupby(['ano_inter']).agg({'morte':'sum'})

losuti_publico =  base[(base['nat_jur'] == 'Administração Pública')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_priv =  base[(base['nat_jur'] == 'Entidades Empresariais')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_ong =  base[(base['nat_jur'] == 'Entidades sem Fins Lucrativos')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()

r = [0,2,4,6,8,10,12,14,16]
r2 = [x+0.5 for x in r]
r3 = [x+1 for x in r]
#g3_bars_bottom = round((losuti_homens/losuti_total)*100)['los_uti']
#g3_bars_top = round((losuti_mulheres/losuti_total)*100)['los_uti']

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)
#p1 = ax1.bar(r, g3_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
#p2 = ax1.bar(r, g3_bars_top, bottom=g3_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')

p1 = ax1.bar(r, round(losuti_publico['los_uti'],1), color = 'darkgreen', width = 0.4, label='Hospital Público')
p2 = ax1.bar(r2, round(losuti_priv['los_uti'],1), color = 'navy',width = 0.4, label='Hospital Privado')
p3 = ax1.bar(r3, round(losuti_ong['los_uti'],1), color = 'orange',width = 0.4, label='Hospital sem Fins Lucrativos')
#ax.bar(X + 0.50, data[2], color = 'r', width = 0.25)

#p1 = sns.barplot(data=losuti_total, x='ano_inter',y='los_uti',hue='Sexo')

#p2 = sns.barplot(data=losuti_mulheres, x='ano_inter',y='los_uti',color = '#DC143C' , label='Homens')

ax1.set_xticks(r2, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
#ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Days', fontweight='bold')
ax1.set_xlabel('Years', fontweight='bold')

ax1.set_title("Average LOS Uti", fontweight='bold')
ax1.bar_label(p1, label_type='edge', fontweight='bold')
ax1.bar_label(p2, label_type='edge', fontweight='bold')
ax1.bar_label(p3, label_type='edge', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.2, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/los_by_hospital.png',dpi=1200)
plt.show()

### LOS por Região

In [ ]:
#losuti_total = base[base['uti']==1].groupby(['ano_inter','sexo']).agg({'los_uti':'mean'}).reset_index()
#losuti_total['Sexo'] = losuti_total['sexo'].apply(lambda x: 'Homem' if x == 1 else "Mulher")

#mortes_hosppublico =  base[base['nat_jur'] == 'Administração Pública'].groupby(['ano_inter']).agg({'morte':'sum'})
#mortes_hosppart =  base[base['nat_jur'] == ].groupby(['ano_inter']).agg({'morte':'sum'})

losuti_S =  base[(base['regiao'] == 'S')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_N =  base[(base['regiao'] == 'N')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_NE =  base[(base['regiao'] == 'NE')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_SE =  base[(base['regiao'] == 'SE')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()
losuti_CO =  base[(base['regiao'] == 'CO')&(base['uti']==1)].groupby(['ano_inter']).agg({'los_uti':'mean'}).reset_index()

r = [0,6,12,18,24,30,36,42,48]
r2 = np.array(r) + 1
r3 = np.array(r2) + 1
r4 = np.array(r3) + 1
r5 = np.array(r4) + 1
#r2 = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5]
#g3_bars_bottom = round((losuti_homens/losuti_total)*100)['los_uti']
#g3_bars_top = round((losuti_mulheres/losuti_total)*100)['los_uti']

fig, ax = plt.subplots(1,1,sharex=True,sharey=True,)
#p1 = ax1.bar(r, g3_bars_bottom, edgecolor='white', color = '#380282' , label='Homens')
#p2 = ax1.bar(r, g3_bars_top, bottom=g3_bars_bottom, edgecolor='white', color = '#DC143C', label='Mulheres')

p1 = ax.bar(r, round(losuti_S['los_uti'],1), color = '#442288', width = 0.9, label='Sul')
p2 = ax.bar(r2, round(losuti_N['los_uti'],1), color = '#6CA2EA',width = 0.9, label='Norte')
p3 = ax.bar(r3, round(losuti_NE['los_uti'],1), color = '#FED23F',width = 0.9, label='Nordeste')
p4 = ax.bar(r4, round(losuti_SE['los_uti'],1), color = '#B5D33D',width = 0.9, label='Sudeste')
p5 = ax.bar(r5, round(losuti_CO['los_uti'],1), color = '#EB7D5B',width = 0.9, label='Centro Oeste')

ax.set_xticks(r3, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
#ax1.set_yticks([0,25,50,75,100],['0%','25%','50%','75%','100%'])
ax.set_ylabel('Days', fontweight='bold')
ax.set_xlabel('Years', fontweight='bold')
ax.set_title("Average LOS Uti", fontweight='bold')
ax.bar_label(p1, label_type='edge', fontweight='bold')
ax.bar_label(p2, label_type='edge', fontweight='bold')
ax.bar_label(p3, label_type='edge', fontweight='bold')
ax.bar_label(p4, label_type='edge', fontweight='bold')
ax.bar_label(p5, label_type='edge', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax.get_legend_handles_labels()
fig.set_figheight(8)
fig.set_figwidth(20)
fig.legend(handles, labels, loc = (0.45, 0.01), ncol=2, labelspacing=0.)
plt.savefig('../images/los_by_region.png',dpi=1200)
plt.show()

### Uso de UTI por tipo de hospital

In [ ]:
# uti_idade = base[base['uti']==1].groupby(['ano_inter','idade_grupo_who']).agg({'uti':'sum'}).reset_index()

uti_hosppublico = base[base['nat_jur'] == 'Administração Pública'].groupby(['ano_inter']).agg({'uti':'sum'}).reset_index()
uti_hosppart = base[base['nat_jur'] == 'Entidades Empresariais'].groupby(['ano_inter']).agg({'uti':'sum'}).reset_index()
uti_hospong = base[base['nat_jur'] == 'Entidades sem Fins Lucrativos'].groupby(['ano_inter']).agg({'uti':'sum'}).reset_index()

r = [0,2,4,6,8,10,12,14,16]
r2 = [x+0.5 for x in r]
r3 = [x+1 for x in r]

fig, ax1 = plt.subplots(1,1,sharex=True,sharey=True,)

p1 = ax1.bar(r, round((uti_hosppublico["uti"]/admissoes_hosppublico.reset_index()["id"])*100,1), color = 'darkgreen', width = 0.4, label='Hospital Público')
p2 = ax1.bar(r2, round((uti_hosppart["uti"]/admissoes_hosppart.reset_index()["id"])*100,1), color = 'navy',width = 0.4, label='Hospital Privado')
p3 = ax1.bar(r3, round((uti_hospong["uti"]/admissoes_hospong.reset_index()["id"])*100,1), color = 'orange',width = 0.4, label='Hospital sem Fins Lucrativos')

ax1.set_xticks(r2, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
# ax1.set_yticks([0,5,10,5,100],['0%','25%','50%','75%','100%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_xlabel('Years', fontweight='bold')

ax1.set_title("Proportion of ICU", fontweight='bold')
ax1.bar_label(p1, label_type='edge', fontweight='bold')
ax1.bar_label(p2, label_type='edge', fontweight='bold')
ax1.bar_label(p3, label_type='edge', fontweight='bold')

fig.set_facecolor("#E5F2A5")
handles, labels = ax1.get_legend_handles_labels()
fig.set_figheight(6)
fig.set_figwidth(10)
fig.legend(handles, labels, loc = (0.2, 0.01), ncol=3, labelspacing=0.)
plt.savefig('../images/uti_by_hospital.png',dpi=1200)
plt.show()

### Uso de uti por idade e ano

In [ ]:
from numpy import append


uti_idade = base[base['uti']==1].groupby(['ano_inter','idade_grupo_who']).agg({'uti':'sum'}).reset_index()

r = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
r2 = [x+0.5 for x in r]
r3 = [x+1 for x in r]
# print(uti_idade[uti_idade["ano_inter"]==2011])

fig, (ax1,ax2,ax3) = plt.subplots(3,1,sharey=True,)

p1 = ax1.bar(r,uti_idade[uti_idade["ano_inter"]==2011]["uti"], width =1, edgecolor="black", color = 'darkgreen', label="2011")
p2 = ax2.bar(r, uti_idade[uti_idade["ano_inter"]==2015]["uti"], width =1, edgecolor="black", color = 'navy',label="2015")
p3 = ax3.bar(r, uti_idade[uti_idade["ano_inter"]==2019]["uti"], width =1, edgecolor="black", color = 'orange',label="2019")

ax1.set_xticks(r,uti_idade[uti_idade["ano_inter"]==2011]["idade_grupo_who"])
ax2.set_xticks(r,uti_idade[uti_idade["ano_inter"]==2015]["idade_grupo_who"])
ax3.set_xticks(r,uti_idade[uti_idade["ano_inter"]==2019]["idade_grupo_who"])

ax1.set_ylabel('Use of ICU in 2011', fontweight='bold')
ax1.set_xlabel('idade_grupo_who', fontweight='bold')

ax2.set_ylabel('Use of ICU in 2015', fontweight='bold')
ax2.set_xlabel('idade_grupo_who', fontweight='bold')

ax3.set_ylabel('Use of ICU in 2019', fontweight='bold')
ax3.set_xlabel('idade_grupo_who', fontweight='bold')


ax1.set_title("Use of ICU by age", fontweight='bold')
ax1.bar_label(p1, label_type='edge', fontweight='bold')
ax2.bar_label(p2, label_type='edge', fontweight='bold')
ax3.bar_label(p3, label_type='edge', fontweight='bold')

fig.set_facecolor("#E5F2A5")
fig.set_figheight(15)
fig.set_figwidth(10)
plt.savefig('../images/use_of_uti_by_age.png',dpi=1200)
plt.show()

### Taxa de letalidade UTI

In [ ]:
dflet_uti = base[base['uti']==1].groupby(['ano_inter','idade_grupo_who']).agg({'id':'count','morte':'sum'}).reset_index()
dflet_uti.columns = ['ano_inter','idade_grupo_who','admissoes','num']
#df_merged = pd.merge(pop_ref, df,  how='left', left_on=['year','idade_grupo_who'], right_on=['ano_inter','idade_grupo_who']) # junta a quantidade por ano por faixa etária com a população de referencia
dflet_uti['taxa'] = dflet_uti['num']/dflet_uti['admissoes'] # calcula a taxa
dfletalidade_uti = pd.merge(dflet_uti, dfWHO,  how='left', left_on=['idade_grupo_who'], right_on=['age_group']) # junta o dataframe com a proporção do WHO
dfletalidade_uti['obitos_esp'] = dfletalidade_uti['taxa']*dfletalidade_uti['world_avg_dec'] # calcula a proporção da quantidade esperados
dfletalidade_uti = dfletalidade_uti.groupby(['ano_inter']).agg({'num':'sum','obitos_esp':'sum','admissoes':'sum','world_avg_dec':'sum'}).reset_index() # agrupa por ANO
# tx = obts esp / pop
#somar as faixas etarias do ano e dividir pela pop de ref total
dfletalidade_uti['taxa_ajustada'] = dfletalidade_uti['obitos_esp']/dfletalidade_uti['world_avg_dec']
dfletalidade_uti['taxa_ajustada_100mil'] = (dfletalidade_uti['taxa_ajustada']*100000).apply(np.ceil)
dfletalidade_uti['taxa_bruta_100mil'] = (dfletalidade_uti['num']/dfletalidade_uti['admissoes'])*100000
dfletalidade_uti

In [ ]:
tx_let_uti = round((dfletalidade_uti['taxa_ajustada'])*100,4)

r = [0,1,2,3,4,5,6,7,8]

fig, ax1 = plt.subplots(1,1,sharex=True,)
p1 = ax1.plot(r, tx_let_uti, marker='o', markerfacecolor='#380282', markersize=4, color = '#380282' , label='Mortality rate')

ax1.set_xticks(r, ["2011","2012","2013","2014","2015","2016","2017","2018","2019"])
ax1.set_yticks([0,10,20,30,40,50,60],['0%','10%','20%','30%','40%','50%','60%'])
ax1.set_ylabel('Proportion', fontweight='bold')
ax1.set_title("Lethality Rate", fontweight='bold')
a = [1,-1,1,-1,1,-1,1,-1,1,-1]

for i in range(len(tx_let_uti)):
    ax1.text(r[i], tx_let_uti[i]+(a[i])*2.2, "%.4f" %tx_let_uti[i], ha="center", fontweight='bold')
    
fig.set_facecolor("#E5F2A5")

fig.set_figheight(6)
fig.set_figwidth(12)
plt.savefig('../images/letality_ratio_uti.png',dpi=1200)
plt.show()

# Análise da Base SIM

In [ ]:
base.head()

In [ ]:
base.columns

In [ ]:
base_mortalidade = base[['estado','regiao','sexo','raca_cor','idade_real_anos','los_hosp','los_uti','uti','diag_princ_trim_upper','nat_jur','morte']]

In [ ]:
base_losuti = base[base['uti']==1][['estado','regiao','sexo','raca_cor','idade_real_anos','diag_princ_trim_upper','nat_jur','los_uti']]

In [ ]:
base_mortalidade.to_csv('../data/base_mortalidade.csv')
base_losuti.to_csv('../data/base_losuti.csv')

In [ ]:
sim2020 = pd.read_csv('../data/Mortalidade_Geral_2020.csv',sep=';')

In [ ]:
sim2020

In [ ]:
sim2020.columns